# 03 - Federated Learning

## Defines

Define the available types of federated learning.

In [4]:
### THIS SECTION NEEDS TO BE SET TO DETERMINE WHICH CONFIGURATION METHOD TO UTILISE

SPLIT_AVAILABLE_METHODS = ['INDIVIDUAL_ATTACK', 'ATTACK_GROUP', 'STRATIFIED']
METHOD = 'STRATIFIED'
NUM_OF_STRATIFIED_CLIENTS = 10  # only applies to stratified method
NUM_OF_ROUNDS = 10              # Number of FL rounds


Include the defines for the dataframe columns and the attack labels and their mappings

In [5]:
from enum import Enum
from includes import dict_34_classes, dict_8_classes, dict_7_classes, dict_2_classes, X_columns, y_column, Colours


##  Imports

In [6]:
%%capture
%pip install flwr[simulation] torch torchvision matplotlib sklearn openml

In [7]:
import os
import pandas as pd
import numpy as np
import flwr as fl
from tqdm import tqdm
import warnings
#warnings.filterwarnings('ignore')

import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision.transforms as transforms
from flwr.common import Metrics
from torch.utils.data import DataLoader, random_split


In [8]:
print("flwr", fl.__version__)
print("numpy", np.__version__)
print("torch", torch.__version__)

DEVICE = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Training on {DEVICE}")

flwr 1.4.0
numpy 1.24.2
torch 1.13.1
Training on cuda:0


## Load the Dataset

In [9]:
DATASET_DIRECTORY = '../datasets/CICIoT2023/'

## Training data

Either read the training pickle file if it exists, or process the dataset from scratch.

In [10]:
# Check to see if the file 'training_data.pkl' exists in the directory. If it does, load it. If not, print an error.
if os.path.isfile('training_data.pkl'):
    print("File exists, loading data...")
    train_df = pd.read_pickle('training_data.pkl')
    print("Training data loaded from pickle file.")

else:
    df_sets = [k for k in os.listdir(DATASET_DIRECTORY) if k.endswith('.csv')]
    df_sets.sort()
    training_sets = df_sets[:int(len(df_sets)*.8)]
    test_sets = df_sets[int(len(df_sets)*.8):]

    # Print the number of files in each set
    print('Training sets: {}'.format(len(training_sets)))
    print('Test sets: {}'.format(len(test_sets)))

    ######################
    # HACK TEMP CODE
    ######################
    # Set training_sets to the last entry of training_sets
    training_sets = training_sets[-1:]
    print(f"HACK TO REPLICATE ORIGINAL AUTHORS CODE WITH ONE FILE TRAIN - {training_sets}")
    ######################
    # HACK END TEMP CODE
    ######################

    # Concatenate all training sets into one dataframe
    dfs = []
    print("Reading training data...")
    for train_set in tqdm(training_sets):
        df_new = pd.read_csv(DATASET_DIRECTORY + train_set)
        dfs.append(df_new)
    train_df = pd.concat(dfs, ignore_index=True)

    # Map y column to the dict_34_classes values - The pickle file already has this done.
    train_df['label'] = train_df['label'].map(dict_34_classes)

    # Save the output to a pickle file
    print("Writing training data to pickle file...")
    train_df.to_pickle('training_data.pkl')

print("Training data size: {}".format(train_df.shape))


File exists, loading data...
Training data loaded from pickle file.
Training data size: (243649, 47)


In [11]:
train_df

,flow_duration,Header_Length,Protocol Type,Duration,Rate,Srate,Drate,fin_flag_number,syn_flag_number,rst_flag_number,...,Std,Tot size,IAT,Number,Magnitue,Radius,Covariance,Variance,Weight,label
0,0.000838,54.62,6.05,64.00,11.961779,11.961779,0.0,0.0,0.0,0.0,...,0.111473,54.45,8.307598e+07,9.5,10.392912,0.037895,0.035900,0.02,141.55,5
1,0.005486,75.88,6.00,64.00,29.502125,29.502125,0.0,0.0,1.0,0.0,...,0.100314,54.24,8.309325e+07,9.5,10.395361,0.143036,0.346802,0.03,141.55,3
2,0.000000,0.00,45.61,65.81,151.517376,151.517376,0.0,0.0,0.0,0.0,...,57.165223,576.80,8.369379e+07,9.5,33.783684,80.958879,8638.780727,0.40,141.55,17
3,0.000000,54.00,6.00,64.00,1.500542,1.500542,0.0,0.0,1.0,0.0,...,0.000000,54.00,8.309408e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,3
4,0.004568,745.42,5.95,65.13,8.082100,8.082100,0.0,0.0,0.0,0.0,...,549.190629,927.04,8.333561e+07,9.5,41.550978,776.661367,318084.344439,0.95,141.55,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
243644,0.000000,54.00,6.00,64.00,19.582485,19.582485,0.0,0.0,0.0,0.0,...,0.000000,54.00,8.331443e+07,9.5,10.392305,0.000000,0.000000,0.00,141.55,2
243645,0.037146,78.22,36.21,63.18,24.542045,24.542045,0.0,0.0,0.0,0.0,...,110.233513,453.78,8.358187e+07,9.5,30.338676,154.660856,23401.960226,0.53,141.55,18
243646,3.293075,1025996.92,17.00,64.00,572.160392,572.160392,0.0,0.0,0.0,0.0,...,0.000000,554.00,8.378910e+07,9.5,33.286634,0.000000,0.000000,0.00,141.55,19
243647,0.047343,35223.00,17.00,64.00,15083.107398,15083.107398,0.0,0.0,0.0,0.0,...,0.000000,50.00,8.309852e+07,9.5,10.000000,0.000000,0.000000,0.00,141.55,4


### Scale the training data input features

In [12]:
scaler = StandardScaler()
train_df[X_columns] = scaler.fit_transform(train_df[X_columns])

---
## Test Data
Concat the test data into a single dataframe

In [13]:
# Check to see if the file 'test_data.pkl' exists in the directory. If it does, load it. If not, print an error.
testing_data_pickle_file = 'testing_data.pkl'

if os.path.isfile(testing_data_pickle_file):
    print(f"File {testing_data_pickle_file} exists, loading data...")
    test_df = pd.read_pickle(testing_data_pickle_file)
    print("Test data loaded from pickle file.")

else:
    print(f"File {testing_data_pickle_file} does not exist, constructing data...")

    df_sets = [k for k in os.listdir(DATASET_DIRECTORY) if k.endswith('.csv')]
    df_sets.sort()
    training_sets = df_sets[:int(len(df_sets)*.8)]
    test_sets = df_sets[int(len(df_sets)*.8):]

    ############################################
    ############################################
    # HACK - Make things quicker for now
    ############################################
    ############################################

    test_sets = df_sets[int(len(df_sets)*.95):]
    
    # Set training_sets to the last entry of training_sets
    test_sets = test_sets[-1:]
    
    ############################################
    ############################################
    # END HACK 
    ############################################
    ############################################

    # Print the number of files in each set
    print('Test sets: {}'.format(len(test_sets)))
    
    # Concatenate all testing sets into one dataframe
    dfs = []
    print("Reading test data...")
    for test_set in tqdm(test_sets):
        df_new = pd.read_csv(DATASET_DIRECTORY + test_set)
        dfs.append(df_new)
    test_df = pd.concat(dfs, ignore_index=True)

    # Map y column to the dict_34_classes values - The pickle file already has this done.
    test_df['label'] = test_df['label'].map(dict_34_classes)

    # Save the output to a pickle file
    print(f"Writing test data to pickle file {testing_data_pickle_file}...")
    test_df.to_pickle(testing_data_pickle_file)

print("Testing data size: {}".format(test_df.shape))

File testing_data.pkl exists, loading data...
Test data loaded from pickle file.
Testing data size: (234745, 47)


### Scale the testing data input features

In [14]:
scaler = StandardScaler()
test_df[X_columns] = scaler.fit_transform(test_df[X_columns])

---
# Define the classification problem - (2 classes, 8 classes or 34 classes)
Change the following cell to select the classification type

In [15]:
binary_classifier = False
group_classifier = False
individual_classifier = True

In [16]:

class_size_map = {2: "Binary", 8: "Group", 34: "Individual"}

if group_classifier:
    print("Group 8 Class Classifier... - Adjusting labels in test and train dataframes")
    # Map y column to the dict_7_classes values
    test_df['label'] = test_df['label'].map(dict_8_classes)
    train_df['label'] = train_df['label'].map(dict_8_classes)
    class_size = "8"
        
elif binary_classifier:
    print("Binary 2 Class Classifier... - Adjusting labels in test and train dataframes")
    # Map y column to the dict_2_classes values
    test_df['label'] = test_df['label'].map(dict_2_classes)
    train_df['label'] = train_df['label'].map(dict_2_classes)
    class_size = "2"

else:
    print ("Individual 34 Class classifier... - No adjustments to labels in test and train dataframes")
    class_size = "34"

Individual 34 Class classifier... - No adjustments to labels in test and train dataframes


---


### Stratify the training dataset

In [17]:
# Define fl_X_train and fl_y_train
fl_X_train = []
fl_y_train = []

# take the train_df[X_columns] and split them into 10 smaller groups in the fl_X_train list using  StratifiedKFold
from sklearn.model_selection import StratifiedKFold
skf = StratifiedKFold(n_splits=NUM_OF_STRATIFIED_CLIENTS, shuffle=True, random_state=42)
for train_index, test_index in skf.split(train_df[X_columns], train_df[y_column]):
    fl_X_train.append(train_df[X_columns].iloc[test_index])
    fl_y_train.append(train_df[y_column].iloc[test_index])

c:\Users\Jon\anaconda3\envs\py310copy\lib\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 5 members, which is less than n_splits=10.
  warnings.warn(


### Convert the training dataset

In [18]:
# Convert the testing daya to X_test and y_test ndarrays
X_test = test_df[X_columns].to_numpy()
y_test = test_df[y_column].to_numpy()

---
### Data check

In [19]:
NUM_OF_CLIENTS = len(fl_X_train)
print("NUM_CLIENTS:", NUM_OF_CLIENTS)

print("NUM_ROUNDS:", NUM_OF_ROUNDS)
print()
print("Original train_df size: {}".format(train_df.shape))

print("Checking training data split groups")
for i in range(len(fl_X_train)):
    print(i, ":", "X Shape", fl_X_train[i].shape, "Y Shape", fl_y_train[i].shape)


# Print the sizes of X_test and y_test
print("\nChecking testing data")
print("X_test size: {}".format(X_test.shape))
print("y_test size: {}".format(y_test.shape))

print("\nDeploy Simulation")

NUM_CLIENTS: 10
NUM_ROUNDS: 10

Original train_df size: (243649, 47)
Checking training data split groups
0 : X Shape (24365, 46) Y Shape (24365,)
1 : X Shape (24365, 46) Y Shape (24365,)
2 : X Shape (24365, 46) Y Shape (24365,)
3 : X Shape (24365, 46) Y Shape (24365,)
4 : X Shape (24365, 46) Y Shape (24365,)
5 : X Shape (24365, 46) Y Shape (24365,)
6 : X Shape (24365, 46) Y Shape (24365,)
7 : X Shape (24365, 46) Y Shape (24365,)
8 : X Shape (24365, 46) Y Shape (24365,)
9 : X Shape (24364, 46) Y Shape (24364,)

Checking testing data
X_test size: (234745, 46)
y_test size: (234745,)

Deploy Simulation


----
# Federated Learning
## Import the libraries and print the versions

In [20]:
import os
import flwr as fl
import numpy as np
import tensorflow as tf

# Make TensorFlow log less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout

label = train_df[y_column]

Define the Client and Server code

In [27]:
import os
import flwr as fl
import numpy as np
import tensorflow as tf

print('scikit-learn {}.'.format(sklearn.__version__))
print("flwr", fl.__version__)
print("numpy", np.__version__)
print("tf", tf.__version__)
# Make TensorFlow log less verbose
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Dropout

import datetime

class NumpyFlowerClient(fl.client.NumPyClient):
    def __init__(self, cid, model, train_data, train_labels):
        self.model = model
        self.cid = cid
        self.train_data = train_data
        self.train_labels = train_labels

    def get_parameters(self, config):
        return self.model.get_weights()

    def fit(self, parameters, config):
        self.model.set_weights(parameters)
        print ("Client ", self.cid, "Training...")
        self.model.fit(self.train_data, self.train_labels, epochs=10, batch_size=64)
        print ("Client ", self.cid, "Training complete...")
        return self.model.get_weights(), len(self.train_data), {}

    def evaluate(self, parameters, config):
        self.model.set_weights(parameters)
        print ("Client ", self.cid, "Evaluating...")
        loss, accuracy = self.model.evaluate(self.train_data, self.train_labels, batch_size=64)
        print(f"{Colours.YELLOW.value}Client {self.cid} evaluation complete - Accuracy: {accuracy:.6f}, Loss: {loss:.6f}{Colours.NORMAL.value}")

        return loss, len(self.train_data), {"accuracy": accuracy}
    
    def predict(self, incoming):
        prediction = np.argmax( self.model.predict(incoming) ,axis=1)
        return prediction

def client_fn(cid: str) -> NumpyFlowerClient:
    """Create a Flower client representing a single organization."""

    # Load model
    #model = tf.keras.applications.MobileNetV2((32, 32, 3), classes=10, weights=None)
    #model.compile("adam", "sparse_categorical_crossentropy", metrics=["accuracy"])

    print ("Client ID:", cid)

    model = Sequential([
      #Flatten(input_shape=(79,1)),
      Flatten(input_shape=(fl_X_train[0].shape[1] , 1)),
      Dense(50, activation='relu'),  
      Dense(25, activation='relu'),  
      Dense(len(label.unique()), activation='softmax')
    ])
    
    model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

   
    partition_id = int(cid)
    X_train_c = fl_X_train[partition_id]
    y_train_c = fl_y_train[partition_id]

    # Create a  single Flower client representing a single organization
    return NumpyFlowerClient(cid, model, X_train_c, y_train_c)


from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
eval_count = 0

def get_evaluate_fn(server_model):
    global eval_count
    """Return an evaluation function for server-side evaluation."""
    # The `evaluate` function will be called after every round
    
    
    def evaluate(server_round, parameters, config):
        global eval_count
        
        # Update model with the latest parameters
        server_model.set_weights(parameters)
        print (f"Server Evaluating... Evaluation Count:{eval_count}")
        loss, accuracy = server_model.evaluate(X_test, y_test)
        
        y_pred = server_model.predict(X_test)
        print ("Prediction: ", y_pred, y_pred.shape)
        #cmatrix = confusion_matrix(y_test, np.rint(y_pred))
        #print ("confusion_matrix:", cmatrix, cmatrix.shape)
                        
        print(f"{Colours.YELLOW.value}Server evaluation complete - Accuracy: {accuracy:.4f}, Loss: {loss:.4f}{Colours.NORMAL.value}")
        
        np.save("y_pred-" + str(eval_count) + ".npy", y_pred)
        #np.save("cmatrix-" + str(eval_count) + ".npy", cmatrix)
        eval_count = eval_count + 1
        
        return loss, {"accuracy": accuracy}
    return evaluate



server_model = Sequential([
    #Flatten(input_shape=(79,1)),
    Flatten(input_shape=(fl_X_train[0].shape[1] , 1)),
    Dense(50, activation='relu'),  
    Dense(25, activation='relu'),  
    Dense(len(label.unique()), activation='softmax')
])


server_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Create FedAvg strategy
strategy = fl.server.strategy.FedAvg(
        fraction_fit=1.0,
        fraction_evaluate=0.5,
        min_fit_clients=2, #10,
        min_evaluate_clients=2, #5,
        min_available_clients=2, #10,
        evaluate_fn=get_evaluate_fn(server_model),
        #evaluate_metrics_aggregation_fn=weighted_average,
)

scikit-learn 1.2.0.
flwr 1.4.0
numpy 1.24.2
tf 2.11.0


In [28]:
%%time
print (f"\n{Colours.YELLOW.value} Deploy simulation... {class_size_map[len(label.unique())]} ({class_size}) Classifier\n{Colours.NORMAL.value}")

start_time = datetime.datetime.now()

# Start simulation
fl.simulation.start_simulation(
    client_fn=client_fn,
    num_clients=NUM_OF_CLIENTS,
    config=fl.server.ServerConfig(num_rounds=NUM_OF_ROUNDS),
    strategy=strategy,
)

end_time = datetime.datetime.now()
print("Total time taken: ", end_time - start_time)

INFO flwr 2023-07-08 18:07:58,500 | app.py:146 | Starting Flower simulation, config: ServerConfig(num_rounds=10, round_timeout=None)



 Deploy simulation... Individual (34) Classifier

(launch_and_evaluate pid=31164) 233/381 [=================>............] - ETA: 0s - loss: 0.2189 - accuracy: 0.9247 [repeated 3x across cluster]
(launch_and_evaluate pid=31164) 345/381 [==========================>...] - ETA: 0s - loss: 0.2223 - accuracy: 0.9259
(launch_and_evaluate pid=31164) 173/381 [============>.................] - ETA: 0s - loss: 0.2170 - accuracy: 0.9245 [repeated 5x across cluster]
(launch_and_evaluate pid=31164)  [repeated 15x across cluster]
(launch_and_evaluate pid=31164)   1/381 [..............................] - ETA: 55s - loss: 0.1734 - accuracy: 0.9531 [repeated 5x across cluster]
(launch_and_evaluate pid=31728) 110/381 [=======>......................] - ETA: 0s - loss: 0.2022 - accuracy: 0.9284 [repeated 4x across cluster]
(launch_and_evaluate pid=31164) 381/381 [==============================] - 0s 874us/step - loss: 0.2416 - accuracy: 0.9262 [repeated 5x across cluster]
(launch_and_evaluate pid=31728) 

2023-07-08 18:08:07,357	INFO worker.py:1636 -- Started a local Ray instance.
INFO flwr 2023-07-08 18:08:11,761 | app.py:180 | Flower VCE: Ray initialized with resources: {'memory': 33186653799.0, 'object_store_memory': 16593326899.0, 'node:127.0.0.1': 1.0, 'GPU': 1.0, 'CPU': 24.0}
INFO flwr 2023-07-08 18:08:11,762 | server.py:86 | Initializing global parameters
INFO flwr 2023-07-08 18:08:11,763 | server.py:273 | Requesting initial parameters from one random client
INFO flwr 2023-07-08 18:08:16,070 | server.py:277 | Received initial parameters from one random client
INFO flwr 2023-07-08 18:08:16,071 | server.py:88 | Evaluating initial parameters


Server Evaluating... Evaluation Count:0
(launch_and_get_parameters pid=11452) Client ID: 8
7336/7336 [==============================] - 5s 676us/step


INFO flwr 2023-07-08 18:08:28,983 | server.py:91 | initial parameters (loss, other metrics): 3.6029906272888184, {'accuracy': 0.00281156157143414}
INFO flwr 2023-07-08 18:08:28,984 | server.py:101 | FL starting
DEBUG flwr 2023-07-08 18:08:28,984 | server.py:218 | fit_round 1: strategy sampled 10 clients (out of 10)


...

INFO flwr 2023-07-08 18:11:55,485 | server.py:119 | fit progress: (10, 0.2516759932041168, {'accuracy': 0.947019100189209}, 206.4999181000003)
DEBUG flwr 2023-07-08 18:11:55,486 | server.py:168 | evaluate_round 10: strategy sampled 5 clients (out of 10)


Prediction:  [[7.6719315e-09 5.0311269e-16 3.7394358e-07 ... 2.2753245e-30
  2.2319377e-22 4.1434787e-26]
 [2.8162866e-08 4.8906659e-14 7.1425130e-11 ... 8.8916035e-28
  1.5755419e-23 1.4679720e-24]
 [6.3082403e-08 1.4264050e-03 1.9314557e-09 ... 1.8353800e-18
  8.2982838e-17 8.2105942e-16]
 ...
 [8.7163304e-10 7.5723650e-16 2.3160020e-07 ... 8.1196548e-31
  3.4276685e-23 1.5974264e-26]
 [1.1154936e-07 6.5868871e-10 1.7200784e-11 ... 1.5774367e-22
  6.4321185e-21 2.9614015e-19]
 [2.2104615e-11 1.4375551e-18 9.9999976e-01 ... 0.0000000e+00
  4.3925962e-35 1.8618102e-34]] (234745, 34)
Server evaluation complete - Accuracy: 0.9470, Loss: 0.2517
(launch_and_fit pid=1336) 339/381 [=========================>....] - ETA: 0s - loss: 0.2133 - accuracy: 0.9271 [repeated 27x across cluster]
(launch_and_fit pid=1336) 168/381 [============>.................] - ETA: 0s - loss: 0.2089 - accuracy: 0.9300 [repeated 28x across cluster]
(launch_and_fit pid=14900) 369/381 [============================>.] 

DEBUG flwr 2023-07-08 18:11:56,219 | server.py:182 | evaluate_round 10 received 5 results and 0 failures
INFO flwr 2023-07-08 18:11:56,220 | server.py:147 | FL finished in 207.2351546
INFO flwr 2023-07-08 18:11:56,221 | app.py:218 | app_fit: losses_distributed [(1, 0.45155642882487557), (2, 0.41725539138928613), (3, 0.39083029344210185), (4, 0.3630958344092109), (5, 0.3422653079032898), (6, 0.32378795742988586), (7, 0.3013587437185041), (8, 0.2821817932286431), (9, 0.2539467602968216), (10, 0.22407538652317788)]
INFO flwr 2023-07-08 18:11:56,221 | app.py:219 | app_fit: metrics_distributed_fit {}
INFO flwr 2023-07-08 18:11:56,222 | app.py:220 | app_fit: metrics_distributed {}
INFO flwr 2023-07-08 18:11:56,223 | app.py:221 | app_fit: losses_centralized [(0, 3.6029906272888184), (1, 0.4543115794658661), (2, 0.42306700348854065), (3, 0.39969688653945923), (4, 0.38259148597717285), (5, 0.35876625776290894), (6, 0.3427808880805969), (7, 0.32311421632766724), (8, 0.301243394613266), (9, 0.282

Total time taken:  0:03:57.726124
CPU times: total: 3min 24s
Wall time: 3min 57s


(launch_and_evaluate pid=11452) Client 3 evaluation complete - Accuracy: 0.938108, Loss: 0.221519
